In [41]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.inspection import permutation_importance
from fairlearn.metrics import MetricFrame
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, demographic_parity_ratio 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing 
from aif360.algorithms.inprocessing import AdversarialDebiasing
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import tensorflow as tf
import warnings


pd.set_option("display.max_columns", None)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
if tf.__version__.startswith('2'):
    tf.compat.v1.disable_eager_execution

In [3]:
df = pd.read_csv('car_insurance_claim.csv')

In [4]:
df.describe()

,ID,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE,CLAIM_FLAG
count,1.030200e+04,10302.000000,10295.000000,10302.000000,9754.000000,10302.000000,10302.000000,10302.000000,10302.000000,9663.000000,10302.000000
mean,4.956631e+08,0.169288,44.837397,0.720443,10.474062,33.416424,5.329159,0.800718,1.710153,8.298148,0.266550
std,2.864675e+08,0.506512,8.606445,1.116323,4.108943,15.869687,4.110795,1.154079,2.159015,5.714450,0.442177
min,6.317500e+04,0.000000,16.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,-3.000000,0.000000
25%,2.442869e+08,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.970043e+08,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000,0.000000
75%,7.394551e+08,0.000000,51.000000,1.000000,13.000000,44.000000,7.000000,2.000000,3.000000,12.000000,1.000000
max,9.999264e+08,4.000000,81.000000,5.000000,23.000000,142.000000,25.000000,5.000000,13.000000,28.000000,1.000000


In [5]:
df.head(2)

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban


In [6]:
a = sum(df['CLAIM_FLAG'] == False)
print(a)

7556


In [7]:
b = sum(df['CLAIM_FLAG'] == True)
print(b)

2746


In [8]:
df = df.drop(columns=['ID','BIRTH'],axis=1)
df = df.applymap(lambda x: x.replace('z_', '') if isinstance(x, str) else x)

print(df.head())

   KIDSDRIV   AGE  HOMEKIDS   YOJ   INCOME PARENT1  HOME_VAL MSTATUS GENDER  \
0         0  60.0         0  11.0  $67,349      No        $0      No      M   
1         0  43.0         0  11.0  $91,449      No  $257,252      No      M   
2         0  48.0         0  11.0  $52,881      No        $0      No      M   
3         0  35.0         1  10.0  $16,039      No  $124,191     Yes      F   
4         0  51.0         0  14.0      NaN      No  $306,251     Yes      M   

      EDUCATION    OCCUPATION  TRAVTIME     CAR_USE BLUEBOOK  TIF CAR_TYPE  \
0           PhD  Professional        14     Private  $14,230   11  Minivan   
1   High School   Blue Collar        22  Commercial  $14,940    1  Minivan   
2     Bachelors       Manager        26     Private  $21,970    1      Van   
3   High School      Clerical         5     Private   $4,010    4      SUV   
4  <High School   Blue Collar        32     Private  $15,440    7  Minivan   

  RED_CAR OLDCLAIM  CLM_FREQ REVOKED  MVR_PTS CLM_AMT  C

In [9]:
numerical = [
    'KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME',
    'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM',
    'CLM_FREQ', 'MVR_PTS',  'CAR_AGE'
]

categorical = [
  'PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION',
    'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY'
]

df[categorical] = df[categorical].apply(lambda x: x.fillna(x.mode()[0]))

        
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$','').replace(',',''))
    return x

for col in ['INCOME', 'HOME_VAL', 'BLUEBOOK', 'OLDCLAIM', 'CLM_AMT']:
    df[col] = df[col].apply(clean_currency)


print("Numerical columns:", numerical)
print("Categorical columns:", categorical)

Numerical columns: ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM', 'CLM_FREQ', 'MVR_PTS', 'CAR_AGE']
Categorical columns: ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY']


In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

In [11]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(random_state=42))
])

In [12]:
X = df.drop(columns=['CLAIM_FLAG', 'CLM_AMT'], axis=1)
y = df['CLAIM_FLAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='roc_auc')
print(f'Cross-validation ROC-AUC: {scores.mean()} ± {scores.std()}')

Cross-validation ROC-AUC: 0.8114692760775519 ± 0.016840503120660884


In [13]:
print(df.columns)

Index(['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'PARENT1', 'HOME_VAL',
       'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'TRAVTIME', 'CAR_USE',
       'BLUEBOOK', 'TIF', 'CAR_TYPE', 'RED_CAR', 'OLDCLAIM', 'CLM_FREQ',
       'REVOKED', 'MVR_PTS', 'CLM_AMT', 'CAR_AGE', 'CLAIM_FLAG', 'URBANICITY'],
      dtype='object')


In [14]:
# # Get feature importances (absolute coefficients for logistic regression)
# importances = np.abs(pipeline..coef_[0])
# feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
# feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# print(feature_importance)

In [15]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1512
           1       0.69      0.44      0.54       549

    accuracy                           0.80      2061
   macro avg       0.76      0.69      0.71      2061
weighted avg       0.79      0.80      0.78      2061

ROC-AUC Score: 0.8123235126878114


In [16]:
groups = ['GENDER', 'EDUCATION', 'MSTATUS', 'PARENT1', 'OCCUPATION', 'URBANICITY']
results = []
privileged = {}
for group in groups:
    privileged[group] = {}

In [17]:
def evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group):
    results = []
    for value in X_test[group].unique():
        mask = X_test[group] == value
        if mask.sum() > 0:
            group_y_test = y_test[mask]
            group_y_pred = y_pred[mask]
            group_y_pred_proba = y_pred_proba[mask]
            try:
                roc_auc = roc_auc_score(group_y_test, group_y_pred_proba)
            except ValueError:
                roc_auc = float('nan')
            tn, fp, fn, tp = confusion_matrix(group_y_test, group_y_pred).ravel()

            results.append({
                "Group": group,
                "Value": value,
                "Accuracy": (group_y_test == group_y_pred).mean(),
                "ROC-AUC": roc_auc,
                "TP": tp,
                "TN": tn,
                "FP": fp,
                "FN": fn
            })

    for result in results:
        print(result)
    
    return results

    
for group in groups:
    results = evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group)

{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.8117770767613038, 'ROC-AUC': 0.8168753557199773, 'TP': 113, 'TN': 659, 'FP': 41, 'FN': 138}
{'Group': 'GENDER', 'Value': 'F', 'Accuracy': 0.7882882882882883, 'ROC-AUC': 0.8078115184977022, 'TP': 129, 'TN': 746, 'FP': 66, 'FN': 169}
{'Group': 'EDUCATION', 'Value': 'High School', 'Accuracy': 0.7670068027210885, 'ROC-AUC': 0.8293456071614236, 'TP': 114, 'TN': 337, 'FP': 44, 'FN': 93}
{'Group': 'EDUCATION', 'Value': 'Bachelors', 'Accuracy': 0.8271375464684015, 'ROC-AUC': 0.8290478416705626, 'TP': 53, 'TN': 392, 'FP': 22, 'FN': 71}
{'Group': 'EDUCATION', 'Value': 'Masters', 'Accuracy': 0.8259860788863109, 'ROC-AUC': 0.7397823869432166, 'TP': 16, 'TN': 340, 'FP': 6, 'FN': 69}
{'Group': 'EDUCATION', 'Value': '<High School', 'Accuracy': 0.752442996742671, 'ROC-AUC': 0.7963286713286714, 'TP': 52, 'TN': 179, 'FP': 29, 'FN': 47}
{'Group': 'EDUCATION', 'Value': 'PhD', 'Accuracy': 0.8324873096446701, 'ROC-AUC': 0.7244677011909058, 'TP': 7, 'TN': 157, 

For each group, we calculate its fairness, based on its equalized odds difference, demographic parity difference, demographic parity ratio.

Then, for each group we calculate the privileged and the unprivileged classes, based on the distance each class' value has with the most favoured value in the group.

In [18]:
def evaluate_fairness(y_true, y_pred, sensitive_features, group_name):
    eod = equalized_odds_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    
    dpd = demographic_parity_difference(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    di_ratio = demographic_parity_ratio(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    print(f'\n group is {group_name}')
    print(f"Demographic Parity Ratio: {di_ratio:.4f}")
    print(f"Equalized Odds Difference: {eod:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    
    
    positive_rates = {}
    for group_value in sensitive_features.unique():
        mask = sensitive_features == group_value
        group_y_pred = y_pred[mask]
        positive_rate = group_y_pred.mean()
        positive_rates[group_value] = positive_rate
        print(f"Subgroup: {group_value}, Positive Prediction Rate: {positive_rate:.4f}")
    
    max_rate = max(positive_rates.values())
    min_rate = min(positive_rates.values())
    
    positive_rates = dict(sorted(positive_rates.items(), key=lambda x: x[1]))
    
    
    values = list(positive_rates.values())
    q1, q3 = np.percentile(values, [25, 75])
    iqr = q3 - q1
    threshold = iqr * 0.5 
        
        
    
    
    for group_value, rate in positive_rates.items():
        
        if rate == min_rate or (rate - min_rate <= threshold):
            print(f"--> Privileged Group: {group_value} (Positive Rate: {rate:.4f})")
            
            if 'privileged' in privileged[group_name]:
                privileged[group_name]['privileged'].append(group_value)
            else:
                privileged[group_name]['privileged'] = [group_value]

        elif rate == max_rate:
            
            print(f"--> Unprivileged Group: {group_value} (Positive Rate: {rate:.4f})")
            if 'unprivileged' in privileged[group_name]:
                privileged[group_name]['unprivileged'].append(group_value)
            else:
                privileged[group_name]['unprivileged'] = [group_value]
            
            
            
            
            
            

In [19]:
for group in groups:
    if group not in ['GENDER','URBANINCITY']:
        evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is EDUCATION
Demographic Parity Ratio: 0.1900
Equalized Odds Difference: 0.3625
Demographic Parity Difference: 0.2177
Subgroup: High School, Positive Prediction Rate: 0.2687
Subgroup: Bachelors, Positive Prediction Rate: 0.1394
Subgroup: Masters, Positive Prediction Rate: 0.0510
Subgroup: <High School, Positive Prediction Rate: 0.2638
Subgroup: PhD, Positive Prediction Rate: 0.0660
--> Privileged Group: Masters (Positive Rate: 0.0510)
--> Privileged Group: PhD (Positive Rate: 0.0660)
--> Privileged Group: Bachelors (Positive Rate: 0.1394)
--> Unprivileged Group: High School (Positive Rate: 0.2687)

 group is MSTATUS
Demographic Parity Ratio: 0.4110
Equalized Odds Difference: 0.2048
Demographic Parity Difference: 0.1546
Subgroup: No, Positive Prediction Rate: 0.2625
Subgroup: Yes, Positive Prediction Rate: 0.1079
--> Privileged Group: Yes (Positive Rate: 0.1079)
--> Unprivileged Group: No (Positive Rate: 0.2625)

 group is PARENT1
Demographic Parity Ratio: 0.3108
Equalized Odds 

Based on the above fairness metrics, we will remove gender and marital status from our groups list, for we do not deem them biased enough.

In [20]:
display(df.head(2))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,No,0.0,No,M,PhD,Professional,14,Private,14230.0,11,Minivan,yes,4461.0,2,No,3,0.0,18.0,0,Highly Urban/ Urban
1,0,43.0,0,11.0,91449.0,No,257252.0,No,M,High School,Blue Collar,22,Commercial,14940.0,1,Minivan,yes,0.0,0,No,0,0.0,1.0,0,Highly Urban/ Urban


We see that the columns occupation, parent1, and occupation indicate the presence of bias in our dataset.

In [21]:
print(groups)

['GENDER', 'EDUCATION', 'MSTATUS', 'PARENT1', 'OCCUPATION', 'URBANICITY']


In [22]:
groups.remove('MSTATUS')
groups.remove('GENDER')

In [23]:
del privileged['GENDER']
del privileged['MSTATUS']
for keys,values in privileged.items():
    print(keys,values)

EDUCATION {'privileged': ['Masters', 'PhD', 'Bachelors'], 'unprivileged': ['High School']}
PARENT1 {'privileged': ['No'], 'unprivileged': ['Yes']}
OCCUPATION {'privileged': ['Manager', 'Doctor', 'Lawyer', 'Professional'], 'unprivileged': ['Blue Collar']}
URBANICITY {'privileged': ['Highly Rural/ Rural'], 'unprivileged': ['Highly Urban/ Urban']}


Let us use aif360 for mitigating bias in the education column

In [24]:

for group in groups:
    
    aif_dict = {}

    for element in privileged[group]['privileged']:    
        aif_dict[element] = 1
    
    for element in privileged[group]['unprivileged']:
        aif_dict[element] = 0


    privileged_class = [key for key, value in aif_dict.items() if value == 1]
    unprivileged_class = [key for key, value in aif_dict.items() if value == 0]

    print(f'privileged classes for group {group} are {privileged_class}')
    print(f'unprivileged classes for group {group} are {unprivileged_class}')
    
    #map column values to 0,1 s, based on whether or not the entry is privileged
    #df[group] = df[group].apply(lambda x: 1 if x in(privileged_class) else 0)




encoder = LabelEncoder()

categorical_columns = [
    'MSTATUS', 'GENDER', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'EDUCATION', 'OCCUPATION','PARENT1','URBANICITY'
]

for col in categorical_columns:
    # Fit and transform the column to encode categorical values
    df[col] = encoder.fit_transform(df[col].astype(str))  # Ensure all categories are considered by converting to string

    # Optionally, print the mapping of original values to encoded labels
    print(f"Encoded {col}: {dict(zip(encoder.classes_, range(len(encoder.classes_))))}")





# Collect all unique class values from all columns
all_unique_values = set()

# Collect unique values for each column
for col in categorical_columns:
    all_unique_values.update(df[col].astype(str).unique())

encoder.fit(sorted(all_unique_values)) 


privileged classes for group EDUCATION are ['Masters', 'PhD', 'Bachelors']
unprivileged classes for group EDUCATION are ['High School']
privileged classes for group PARENT1 are ['No']
unprivileged classes for group PARENT1 are ['Yes']
privileged classes for group OCCUPATION are ['Manager', 'Doctor', 'Lawyer', 'Professional']
unprivileged classes for group OCCUPATION are ['Blue Collar']
privileged classes for group URBANICITY are ['Highly Rural/ Rural']
unprivileged classes for group URBANICITY are ['Highly Urban/ Urban']
Encoded MSTATUS: {'No': 0, 'Yes': 1}
Encoded GENDER: {'F': 0, 'M': 1}
Encoded CAR_USE: {'Commercial': 0, 'Private': 1}
Encoded CAR_TYPE: {'Minivan': 0, 'Panel Truck': 1, 'Pickup': 2, 'SUV': 3, 'Sports Car': 4, 'Van': 5}
Encoded RED_CAR: {'no': 0, 'yes': 1}
Encoded REVOKED: {'No': 0, 'Yes': 1}
Encoded EDUCATION: {'<High School': 0, 'Bachelors': 1, 'High School': 2, 'Masters': 3, 'PhD': 4}
Encoded OCCUPATION: {'Blue Collar': 0, 'Clerical': 1, 'Doctor': 2, 'Home Maker': 3

LabelEncoder()

In [25]:
# encoded_df = encode_df(df)
display(df.head(2))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,0,0.0,0,1,4,6,14,1,14230.0,11,0,1,4461.0,2,0,3,0.0,18.0,0,1
1,0,43.0,0,11.0,91449.0,0,257252.0,0,1,2,0,22,0,14940.0,1,0,1,0.0,0,0,0,0.0,1.0,0,1


Check feature importances

In [31]:
importances = np.abs(model.coef_[0])
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance)

       Feature  Importance
22  URBANICITY    2.176261
19     REVOKED    0.807494
12     CAR_USE    0.743553
7      MSTATUS    0.352968
5      PARENT1    0.323293
4       INCOME    0.240328
14         TIF    0.233565
6     HOME_VAL    0.227768
20     MVR_PTS    0.224410
0     KIDSDRIV    0.218072
11    TRAVTIME    0.213465
18    CLM_FREQ    0.200429
15    CAR_TYPE    0.138125
13    BLUEBOOK    0.136032
21     CAR_AGE    0.112820
17    OLDCLAIM    0.081530
2     HOMEKIDS    0.079110
8       GENDER    0.058362
10  OCCUPATION    0.056933
3          YOJ    0.051669
16     RED_CAR    0.030251
1          AGE    0.028408
9    EDUCATION    0.018256


In [26]:
for col in df.columns:
    if df[col].dtype == 'object': 
        print(col)

AIF 360

First we create an AIF360 dataset.

In [ ]:

label_name = 'CLAIM_FLAG'
favorable_classes = [0]
protected_attribute_names = ['EDUCATION', 'PARENT1', 'OCCUPATION']
privileged_classes = [[4,3], [0], [5,4,2]]  

# Optional parameters (if needed)
#categorical_features = ['MSTATUS', 'GENDER', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED']  # Specify categorical features
features_to_drop = ['id', 'address', 'CLM_AMT']  # Columns to drop (if any)
na_values = ['NA', '?', '']  # Handle missing values






df[numerical] = df[numerical].fillna(df[numerical].mean())
df[categorical] = df[categorical].fillna(df[categorical].mode().iloc[0])
scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])





dataset = StandardDataset(
    df.drop('CLM_AMT',axis=1),
    label_name=label_name,
    favorable_classes=favorable_classes,
    protected_attribute_names=protected_attribute_names,
    privileged_classes=privileged_classes,
    # categorical_features=categorical_columns,  # Pass all categorical columns
    # features_to_drop=features_to_drop,
    # na_values=na_values
)

Define privileged and unprivileged groups

In [28]:
unprivileged_groups = [{'EDUCATION': 2}]
privileged_groups = [{'EDUCATION': 4}, {'EDUCATION': 3}, {'EDUCATION': 1}] 

unprivileged_groups.append({'PARENT1': 1})
privileged_groups.append({'PARENT1': 0})

unprivileged_groups.append({'OCCUPATION': 7})
privileged_groups.append({'OCCUPATION': 5})
privileged_groups.append({'OCCUPATION' : 4}) 
privileged_groups.append({'OCCUPATION' : 2}) 


Pre processing mitigation techniques


Reweighting

In [ ]:
reweighing = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

reweighed_data = reweighing.fit_transform(dataset)
X_train, X_test, y_train, y_test = train_test_split(
    reweighed_data.features, reweighed_data.labels.ravel(), test_size=0.3, random_state=42
)

X_test = pd.DataFrame(X_test, columns=X.columns)

model = LogisticRegression(penalty='l2', C=0.1, random_state=42, max_iter=1000)
model.fit(X_train, y_train)




y_pred = model.predict(X_test)

dataset_test = dataset.copy()
dataset_test.features = X_test
dataset_test.labels = y_test

reweighed_dataset_test = dataset.copy()
reweighed_dataset_test.features = X_test
reweighed_dataset_test.labels = y_pred



y_pred_proba = model.predict_proba(X_test)[:, 1]

C:\Users\steve\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\steve\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [30]:
groups.remove('URBANICITY')

for group in groups:
    #evaluate_fairness(y_test, y_pred, X_test[group], group)
    evaluate_accuracy(X_test, y_test, y_pred, y_pred_proba, group)

{'Group': 'EDUCATION', 'Value': 2.0, 'Accuracy': 0.7338618346545867, 'ROC-AUC': 0.8087274061228572, 'TP': 150, 'TN': 498, 'FP': 66, 'FN': 169}
{'Group': 'EDUCATION', 'Value': 3.0, 'Accuracy': 0.8508634222919937, 'ROC-AUC': 0.790770401747303, 'TP': 32, 'TN': 510, 'FP': 11, 'FN': 84}
{'Group': 'EDUCATION', 'Value': 1.0, 'Accuracy': 0.8192352259559675, 'ROC-AUC': 0.7874825269423277, 'TP': 74, 'TN': 633, 'FP': 29, 'FN': 127}
{'Group': 'EDUCATION', 'Value': 4.0, 'Accuracy': 0.8689138576779026, 'ROC-AUC': 0.7849125836391108, 'TP': 10, 'TN': 222, 'FP': 4, 'FN': 31}
{'Group': 'EDUCATION', 'Value': 0.0, 'Accuracy': 0.7596371882086168, 'ROC-AUC': 0.7976781982818668, 'TP': 71, 'TN': 264, 'FP': 31, 'FN': 75}
{'Group': 'PARENT1', 'Value': 0.0, 'Accuracy': 0.7990377498149519, 'ROC-AUC': 0.7991477014733597, 'TP': 216, 'TN': 1943, 'FP': 110, 'FN': 433}
{'Group': 'PARENT1', 'Value': 1.0, 'Accuracy': 0.7840616966580977, 'ROC-AUC': 0.822106388666132, 'TP': 121, 'TN': 184, 'FP': 31, 'FN': 53}
{'Group': 'O

How about resampling?

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

smote = SMOTE(random_state=42)

pipeline = ImbPipeline([
    ("preprocessor", preprocessor),
    ("resampler", smote),
    ("classifier", LogisticRegression(random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

for group in groups:
    evaluate_fairness(y_test,y_pred, X_test[group], group)

Accuracy: 0.7269492073762537

 group is EDUCATION
Demographic Parity Ratio: 0.3627
Equalized Odds Difference: 0.3035
Demographic Parity Difference: 0.3685
Subgroup: 2, Positive Prediction Rate: 0.5504
Subgroup: 3, Positive Prediction Rate: 0.2747
Subgroup: 1, Positive Prediction Rate: 0.3673
Subgroup: 4, Positive Prediction Rate: 0.2097
Subgroup: 0, Positive Prediction Rate: 0.5782
--> Privileged Group: 4 (Positive Rate: 0.2097)
--> Privileged Group: 3 (Positive Rate: 0.2747)
--> Unprivileged Group: 0 (Positive Rate: 0.5782)

 group is PARENT1
Demographic Parity Ratio: 0.6073
Equalized Odds Difference: 0.1845
Demographic Parity Difference: 0.2493
Subgroup: 0, Positive Prediction Rate: 0.3856
Subgroup: 1, Positive Prediction Rate: 0.6350
--> Privileged Group: 0 (Positive Rate: 0.3856)
--> Unprivileged Group: 1 (Positive Rate: 0.6350)

 group is OCCUPATION
Demographic Parity Ratio: 0.1953
Equalized Odds Difference: 0.5251
Demographic Parity Difference: 0.4846
Subgroup: 6, Positive Predic

In processing Mitigation Techniques

First, we ll try and test the results of adversial debiasing

In [32]:
unprivileged_groups = [{'EDUCATION': 2, 'PARENT1': 1, 'OCCUPATION': 7}]
privileged_groups = [{'EDUCATION': 4, 'PARENT1': 0, 'OCCUPATION': 5}]
tf.compat.v1.disable_eager_execution()

sess = tf.compat.v1.Session()

train_dataset, test_dataset = dataset.split([0.7], shuffle=True, seed=42)


adversarial_model = AdversarialDebiasing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    scope_name='debiased_classifier',
    sess=sess,
    num_epochs=100,
    batch_size=128,
    classifier_num_hidden_units=100,
    debias=True,
    adversary_loss_weight=0.001
)

adversarial_model.fit(train_dataset)
y_pred_adversarial = adversarial_model.predict(test_dataset).labels

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

epoch 0; iter: 0; batch classifier loss: 0.823118; batch adversarial loss: 1.269651
epoch 1; iter: 0; batch classifier loss: 0.655212; batch adversarial loss: 1.145022
epoch 2; iter: 0; batch classifier loss: 0.667269; batch adversarial loss: 1.206553
epoch 3; iter: 0; batch classifier loss: 0.605778; batch adversarial loss: 1.013282
epoch 4; iter: 0; batch classifier loss: 0.645542; batch adversarial loss: 0.944553
epoch 5; iter: 0; batch classifier loss: 0.587292; batch adversarial loss: 0.771228
epoch 6; iter: 0; batch classifier loss: 0.595542; batch adversarial loss: 0.582338
epoch 7; iter: 0; batch classifier loss: 0.653243; batch adversarial loss: 0.408693
epoch 8; iter: 0; batch classifier loss: 0.583471; batch adversarial loss: 0.316789
epoch 9; iter: 0; batch classifier loss: 0.546354; batch adversarial loss: 0.243634
epoch 10; iter: 0; batch classifier loss: 0.

In [33]:
print("Adversarial Debiasing Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_adversarial))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_adversarial))
print("Classification Report:")
print(classification_report(y_test, y_pred_adversarial))
for group in groups:
    evaluate_fairness(y_test,y_pred_adversarial,X_test[group],group)

Adversarial Debiasing Results:
Accuracy: 0.6285991588482692
ROC-AUC: 0.5069271667084547
Classification Report:
              precision    recall  f1-score   support

         0.0       0.74      0.77      0.75      2268
         1.0       0.28      0.25      0.26       823

    accuracy                           0.63      3091
   macro avg       0.51      0.51      0.51      3091
weighted avg       0.61      0.63      0.62      3091


 group is EDUCATION
Demographic Parity Ratio: 0.8114
Equalized Odds Difference: 0.0606
Demographic Parity Difference: 0.0488
Subgroup: 2.0, Positive Prediction Rate: 0.2435
Subgroup: 3.0, Positive Prediction Rate: 0.2386
Subgroup: 1.0, Positive Prediction Rate: 0.2248
Subgroup: 4.0, Positive Prediction Rate: 0.2097
Subgroup: 0.0, Positive Prediction Rate: 0.2585
--> Privileged Group: 4.0 (Positive Rate: 0.2097)
--> Unprivileged Group: 0.0 (Positive Rate: 0.2585)

 group is PARENT1
Demographic Parity Ratio: 0.9999
Equalized Odds Difference: 0.0105
Demograp

Fairlearn Constrains

First, we will use it for a Demographic Parity Constraint

In [ ]:
lg = LogisticRegression(random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

fair_model = ExponentiatedGradient(
    estimator=lg, 
    constraints=DemographicParity(),
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("fair_classifier", fair_model)
])

X = df.drop(columns=['CLM_AMT', 'CLAIM_FLAG'], axis=1)
y = df['CLAIM_FLAG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

fair_model.fit(X_train, y_train, sensitive_features=X_train[groups])

y_pred = fair_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6670980265286315


In [35]:
for group in groups:
    evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is EDUCATION
Demographic Parity Ratio: 0.6765
Equalized Odds Difference: 0.0892
Demographic Parity Difference: 0.0715
Subgroup: 2, Positive Prediction Rate: 0.1959
Subgroup: 3, Positive Prediction Rate: 0.1821
Subgroup: 1, Positive Prediction Rate: 0.1495
Subgroup: 4, Positive Prediction Rate: 0.2210
Subgroup: 0, Positive Prediction Rate: 0.2063
--> Privileged Group: 1 (Positive Rate: 0.1495)
--> Unprivileged Group: 4 (Positive Rate: 0.2210)

 group is PARENT1
Demographic Parity Ratio: 0.8782
Equalized Odds Difference: 0.0324
Demographic Parity Difference: 0.0250
Subgroup: 0, Positive Prediction Rate: 0.1806
Subgroup: 1, Positive Prediction Rate: 0.2057
--> Privileged Group: 0 (Positive Rate: 0.1806)
--> Unprivileged Group: 1 (Positive Rate: 0.2057)

 group is OCCUPATION
Demographic Parity Ratio: 0.7030
Equalized Odds Difference: 0.2790
Demographic Parity Difference: 0.0653
Subgroup: 6, Positive Prediction Rate: 0.1573
Subgroup: 3, Positive Prediction Rate: 0.1544
Subgroup: 0, 

Now we will evaluate its results for choosing Equalized Odds as a fairness constraint

In [36]:
log_reg = LogisticRegression(random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

fair_model = ExponentiatedGradient(
    estimator=log_reg, 
    constraints=EqualizedOdds(), 
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("fair_classifier", fair_model)
])

X = df.drop(columns=['CLM_AMT', 'CLAIM_FLAG'], axis=1)
y = df['CLAIM_FLAG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

fair_model.fit(X_train, y_train, sensitive_features=X_train[groups])

y_pred = fair_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\fairlearn\reductions\_moments\utility_parity.py:212: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.pos_basis[i] = 0 + zero_vec
c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\fairlearn\reductions\_moments\utility_parity.py:213: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.neg_basis[i] = 0 + zero_vec
c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\fairlearn\reductions\_moments\utility_parity.py:212: P

Accuracy: 0.6324813976059528


In [37]:
for group in groups:
    evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is EDUCATION
Demographic Parity Ratio: 0.8702
Equalized Odds Difference: 0.0929
Demographic Parity Difference: 0.0313
Subgroup: 2, Positive Prediction Rate: 0.2197
Subgroup: 3, Positive Prediction Rate: 0.2245
Subgroup: 1, Positive Prediction Rate: 0.2410
Subgroup: 4, Positive Prediction Rate: 0.2097
Subgroup: 0, Positive Prediction Rate: 0.2404
--> Privileged Group: 4 (Positive Rate: 0.2097)
--> Privileged Group: 2 (Positive Rate: 0.2197)
--> Unprivileged Group: 1 (Positive Rate: 0.2410)

 group is PARENT1
Demographic Parity Ratio: 0.8863
Equalized Odds Difference: 0.0326
Demographic Parity Difference: 0.0264
Subgroup: 0, Positive Prediction Rate: 0.2321
Subgroup: 1, Positive Prediction Rate: 0.2057
--> Privileged Group: 1 (Positive Rate: 0.2057)
--> Unprivileged Group: 0 (Positive Rate: 0.2321)

 group is OCCUPATION
Demographic Parity Ratio: 0.7345
Equalized Odds Difference: 0.1645
Demographic Parity Difference: 0.0684
Subgroup: 6, Positive Prediction Rate: 0.2230
Subgroup: 3